In [70]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
from sophius.db import database, Experiments, Models, ModelEpochs
import sqlite3
import pandas as pd

with database:
    database.create_tables([Experiments, Models, ModelEpochs])

with sqlite3.connect('../data/models.db') as conn:
    exp_df = pd.read_sql('SELECT * FROM experiments', conn)
    exp_df.id += 1
    exp_df.drop(columns='hash', inplace=True)
    for col in ['opt_params', 'sch_params']:
        exp_df[col] = exp_df[col].apply(eval)
    
    models_df = pd.read_sql('SELECT * FROM models', conn)
    models_df.id += 1
    models_df.exp_id = models_df.exp_id.astype(int) + 1
    
    numeric_cols = models_df.columns.tolist()
    numeric_cols.remove('hash')
    for col in numeric_cols:
        models_df[col] = pd.to_numeric(models_df[col], errors='coerce')        
    for col in ['val_acc', 'train_acc']:
        models_df[col] = models_df[col].round(4)
    models_df.time = models_df.time.round(1)
    
    epochs_df = pd.read_sql('SELECT * FROM model_epochs', conn)
    for col in epochs_df.columns:
        epochs_df[col] = pd.to_numeric(epochs_df[col], errors='coerce')
    epochs_df.exp_id += 1
    epochs_df.model_id += 1    
    epochs_df.epoch = epochs_df.epoch.astype(int)
    for col in ['val_acc', 'train_acc']:
        epochs_df[col] = epochs_df[col].round(4)
    epochs_df.loss = epochs_df.loss.round(2)
    epochs_df.time = epochs_df.time.round(1)
    
exp_df

,id,val_size,batch_size,num_epoch,random_seed,optimizer,opt_params,scheduler,sch_params,in_shape,out_shape
0,1,10000,256,50,42,AdamW,{'lr': 0.001},ExponentialLR,{'gamma': 0.95},"(3, 32, 32)",10


In [76]:
exp_params = exp_df.to_dict(orient='records')
exp_params

[{'id': 1,
  'val_size': 10000,
  'batch_size': 256,
  'num_epoch': 50,
  'random_seed': 42,
  'optimizer': 'AdamW',
  'opt_params': {'lr': 0.001},
  'scheduler': 'ExponentialLR',
  'sch_params': {'gamma': 0.95},
  'in_shape': '(3, 32, 32)',
  'out_shape': '10'}]

In [78]:
print('Insert experiments')    
for exp_params in exp_df.to_dict(orient='records'):    
    Experiments.insert(**exp_params).on_conflict_ignore().execute()

Insert experiments


In [69]:
from tqdm import tqdm

print('Insert experiments')    
for exp_params in exp_df.to_dict(orient='records'):    
    Experiments.insert(**exp_params).on_conflict_ignore().execute()
    
print('Insert models')    
Models.insert_many(models_df.to_dict(orient='records')).on_conflict_ignore().execute()    
print('Insert epochs')
for i in tqdm(models_df.id.unique()):
    mask = epochs_df.model_id == i
    ModelEpochs.insert_many(epochs_df[mask].to_dict(orient='records')).on_conflict_ignore().execute()

Insert experiments
Insert models
Insert epochs


100%|██████████| 8531/8531 [00:49<00:00, 172.86it/s]
